In [1]:
import sys
import os
import subprocess
import glob
import numpy as np
import wavio
import time;
import random;
from common import opts;
import common.utils as U;

In [2]:
import argparse

In [3]:
src_wav_dir = "./datasets/before_processed_audio/"
train_dest_wav_dir = ".//datasets/processing_tmp_audios/train/"
test_dest_wav_dir = "./datasets/processing_tmp_audios/test/"

In [4]:
def getOpts():
    parser = argparse.ArgumentParser(description='Transfer Learning for ACDNet');
    parser.add_argument('--netType', default='ACDNet_TL_Model_Extend',  required=False);
    parser.add_argument('--data', default='../datasets/processed/',  required=False);
    parser.add_argument('--dataset', required=False, default='uec_iot', choices=['10']);
    parser.add_argument('--BC', default=True, action='store_true', help='BC learning');
    parser.add_argument('--strongAugment', default=True,  action='store_true', help='Add scale and gain augmentation');
    #在ipynb中，不能使用parser.parse，要改用parser.parse_known_args()
    opt, unknown = parser.parse_known_args()
    #Leqarning settings
    opt.batchSize = 88;
    opt.weightDecay = 5e-4;
    opt.momentum = 0.09;
    opt.nEpochs = 10;#2000;
    opt.LR = 0.1;
    opt.schedule = [0.3, 0.6, 0.9];
    opt.warmup = 10;
    opt.device = 'cpu';#torch.device("cuda:0" if torch.cuda.is_available() else "cpu");
    #Basic Net Settings
    opt.nClasses = 12#50;
    opt.nFolds = 5;
    opt.splits = [i for i in range(1, opt.nFolds + 1)];
    opt.sr = 20000;
    opt.inputLength = 30225;
    #Test data
    opt.nCrops = 4;
    return opt

In [5]:
def convert_sr(src_path, dst_path, sr):
    # print('* {} -> {}'.format(src_path, dst_path))
    if not os.path.exists(dst_path):
        os.mkdir(dst_path);
    for src_file in sorted(glob.glob(os.path.join(src_path, '*.wav'))):
        dst_file = src_file.replace(src_path, dst_path);
        subprocess.call('ffmpeg -i {} -ac 1 -ar {} -loglevel error -y {}'.format(
            src_file, sr, dst_file), shell=True);

In [6]:
def main():
    classes_dict = {
        17:"17_pouring_water", #pouring_water
        18:"18_toilet_flushing", #toilet_flushing
        21:"21_sneezing", #snezzing
        24:"24_coughing", #coughing
        51:"51_kettle_sound", #kettle_sound
        52:"52_alarm", #alarm
        #53:"53_boiling_water_bubble_sound", #boiling_water_bubble_sound
        54:"54_ringtone", #rington
        55:"55_shower_water", #shower_water
        56:"56_pain_sounds", #pain_sounds
        57:"57_footsteps", #footsteps
        98:"98_silence", #silence
        99:"99_other_sounds", #other_sounds
                   };
    sr = 20000;
    iter_count = 0;
    for k in classes_dict:
        cur_src_dir = os.path.join(src_wav_dir,classes_dict[k]);
        print(f"current work directory:{cur_src_dir}\n");
        for w in sorted(glob.glob(os.path.join(cur_src_dir, '*.wav'))):
            fname = "{}.wav".format(os.path.basename(w).split('.')[0]);
            dest_fname = os.path.join(dest_wav_dir,classes_dict[k],fname);
            print(f"Convert Sampling Rate:{w} >> {dest_fname}");
            subprocess.call('ffmpeg -i {} -ac 1 -ar {} -loglevel error -y {}'.format(
            w, sr, dest_fname), shell=True);
            # print(f"wav:{w}");
        # print(f"key:{k}, value:{classes_dict[k]}");
    

In [7]:
# main()

In [8]:
def create_dataset(src_path, dst_path):
    # print('* {} -> {}'.format(src_path, dst_path))
    classes_dict = {
        17:"17_pouring_water", #pouring_water
        18:"18_toilet_flushing", #toilet_flushing
        21:"21_sneezing", #snezzing
        24:"24_coughing", #coughing
        51:"51_kettle_sound", #kettle_sound
        52:"52_alarm", #alarm
        #53:"53_boiling_water_bubble_sound", #boiling_water_bubble_sound
        54:"54_ringtone", #rington
        55:"55_shower_water", #shower_water
        56:"56_pain_sounds", #pain_sounds
        57:"57_footsteps", #footsteps
        98:"98_silence", #silence
        99:"99_other_sounds", #other_sounds
    };
    # idx_dict = {
    #     17:1, #pouring_water
    #     18:2, #toilet_flushing
    #     21:3, #snezzing
    #     24:4, #coughing
    #     51:5, #kettle_sound
    #     52:6, #alarm
    #     #53:"53_boiling_water_bubble_sound", #boiling_water_bubble_sound
    #     54:7, #rington
    #     55:8, #shower_water
    #     56:9, #pain_sounds
    #     57:10, #footsteps
    #     98:11, #silence
    #     99:12, #other_sounds
    # };
    my_dataset = {};
    for fold in range(1, 6):
        print(f"--Start to preparing fold{fold} dataset...---------------");
        my_dataset['fold{}'.format(fold)] = {}
        my_sounds = []
        my_labels = []
        for k in classes_dict:
            cur_src_dir = os.path.join(src_path,classes_dict[k]);
            print(f"current source directory:{cur_src_dir}");
            for wav_file in sorted(glob.glob(os.path.join(cur_src_dir, '*.wav'))):
                sound = wavio.read(wav_file).data.T[0]
                start = sound.nonzero()[0].min()
                end = sound.nonzero()[0].max()
                sound = sound[start: end + 1]  # Remove silent sections
                # label = k;#int(os.path.splitext(wav_file)[0].split('-')[-1])
                my_sounds.append(sound)
                my_labels.append(k)
                print(f"sound:{wav_file}\nlabel:{k}") 
        print(f"--End of preparing fold{fold} dataset-------------------");

        my_dataset['fold{}'.format(fold)]['sounds'] = my_sounds
        my_dataset['fold{}'.format(fold)]['labels'] = my_labels

    np.savez(dst_path, **my_dataset)

In [9]:
# save_npz_folder = "../../../RLRepo/Works/Projects/TransferLearning_for_ACDNet/datasets/fsd50k_processed_audios/test_fsd50_20K.npz"
# create_dataset(test_dest_wav_dir,save_npz_folder);

In [10]:
def load_train_npz(train_npz):
    dataset = np.load(train_npz, allow_pickle=True);
    print(f"len of sound:{len(dataset['fold1'].item()['sounds'][123])}");
    print(f"label:{dataset['fold1'].item()['labels'][123]}");
    # print(f"sound:{dataset['fold3'].item()['sounds'][123]}");
    # print(f"label:{dataset['fold3'].item()['labels'][123]}");
    # train_sounds = []
    # train_labels = []
    # for i in range(1, opt.nFolds + 1):
    #     sounds = dataset['fold{}'.format(i)].item()['sounds']
    #     labels = dataset['fold{}'.format(i)].item()['labels']
    #     if i != split:
    #         train_sounds.extend(sounds)
    #         train_labels.extend(labels)

In [11]:
#load_train_npz("../../../RLRepo/Works/Projects/TransferLearning_for_ACDNet/datasets/fsd50k_processed_audios/train_fsd50_20K__202401041450.npz")

In [20]:
class ValGenerator():
    #Generates data for Keras
    def __init__(self, samples, labels, options):
        random.seed(42);
        #Initialization
        print(len(samples));
        self.data = [(samples[i], labels[i]) for i in range (0, len(samples))];
        self.opt = options;
        self.batch_size = 88;#options.batchSize // options.nCrops;
        self.preprocess_funcs = self.preprocess_setup();
        self.map_dict= {
            17:1, #pouring_water
            18:2, #toilet_flushing
            21:3, #snezzing
            24:4, #coughing
            51:5, #kettle_sound
            52:6, #alarm
            #53:"53_boiling_water_bubble_sound", #boiling_water_bubble_sound
            54:7, #rington
            55:8, #shower_water
            56:9, #pain_sounds
            57:10, #footsteps
            98:11, #silence
            99:12, #other_sounds
        };

    def get_data(self):
        #Generate one batch of data
        x, y = self.generate();
        x = np.expand_dims(x, axis=1)
        x = np.expand_dims(x, axis=3)
        print(x.shape);
        print(y.shape);
        return x, y

    def generate(self):
        #Generates data containing batch_size samples
        sounds = [];
        labels = [];
        indexes = None;
        for i in range(self.batch_size):
            sound, target = self.data[i];
            target = self.map_dict[target] - 1;
            sound = self.preprocess(sound).astype(np.float32)
            label = np.zeros((self.opt.nCrops, self.opt.nClasses));
            label[:,target] = 1;

            sounds.append(sound);
            labels.append(label);

        sounds = np.asarray(sounds);
        labels = np.asarray(labels);
        print(f"sounds shape:{sounds.shape}")
        sounds = sounds.reshape(sounds.shape[0]*sounds.shape[1], sounds.shape[2]);
        labels = labels.reshape(labels.shape[0]*labels.shape[1], labels.shape[2]);

        return sounds, labels;

    def preprocess_setup(self):
        funcs = []
        funcs += [U.padding(self.opt.inputLength // 2),
                  U.normalize(32768.0),
                  U.multi_crop(self.opt.inputLength, self.opt.nCrops)]

        return funcs

    def preprocess(self, sound):
        for f in self.preprocess_funcs:
            sound = f(sound)

        return sound;



In [21]:
def create_test_data_src_npz(src_path, dst_path):
    # print('* {} -> {}'.format(src_path, dst_path))
    classes_dict = {
        17:"17_pouring_water", #pouring_water
        18:"18_toilet_flushing", #toilet_flushing
        21:"21_sneezing", #snezzing
        24:"24_coughing", #coughing
        51:"51_kettle_sound", #kettle_sound
        52:"52_alarm", #alarm
        #53:"53_boiling_water_bubble_sound", #boiling_water_bubble_sound
        54:"54_ringtone", #rington
        55:"55_shower_water", #shower_water
        56:"56_pain_sounds", #pain_sounds
        57:"57_footsteps", #footsteps
        98:"98_silence", #silence
        99:"99_other_sounds", #other_sounds
    };
    
    my_dataset = {};
    my_sounds = []
    my_labels = []
    my_dataset['testdata'] = {}
    for k in classes_dict:
        cur_src_dir = os.path.join(src_path,classes_dict[k]);
        print(f"current source directory:{cur_src_dir}");
        for wav_file in sorted(glob.glob(os.path.join(cur_src_dir, '*.wav'))):
            sound = wavio.read(wav_file).data.T[0]
            start = sound.nonzero()[0].min()
            end = sound.nonzero()[0].max()
            sound = sound[start: end + 1]  # Remove silent sections
            # label = k;#int(os.path.splitext(wav_file)[0].split('-')[-1])
            my_sounds.append(sound)
            my_labels.append(k)
            print(f"sound:{wav_file}\nlabel:{k}") 
    print(f"--End of preparing test dataset-------------------");

    my_dataset['testdata']['sounds'] = my_sounds
    my_dataset['testdata']['labels'] = my_labels

    np.savez(dst_path, **my_dataset)

In [22]:
# save_test_npz_folder = "./datasets/fsd50k_processed_audios/Src_test_fsd50_20K_data.npz"
# create_test_dataset(test_dest_wav_dir,save_test_npz_folder);

In [23]:
def load_test_npz(test_npz):
    dataset = np.load(test_npz, allow_pickle=True);
    # sound_ary = np.asarray();
    sound_tmp = dataset['testdata'].item()['labels'];
    print(f"len of test sounds is {len(sound_tmp)}")
        # extend(repeat(x, 100))
    # sound_data = []
    # sound_data.extend(sound_tmp, len(sound_tmp))
    # print(dataset['x'])

In [24]:
# load_test_npz("./datasets/fsd50k_processed_audios/Src_test_fsd50_20K_data.npz");

In [25]:
def create_test_npz(test_src_npz=None):
    opt = getOpts();#opts.parse();
    opts.display_info(opt);
    opt.batchSize=88;
    for sr in [20000]:
        opt.sr = sr;
        opt.inputLength = 30225;
        # mainDir = os.getcwd();
        # test_data_dir = os.path.join(mainDir, './datasets/fsd50k_processed_audios/');
        # print(test_data_dir)
        # if not os.path.exists(test_data_dir):
        #     os.mkdir(test_data_dir);

        val_sounds = [];
        val_labels = [];
        dataset = np.load(test_src_npz, allow_pickle=True);
        # for s in opt.splits:
        start_time = time.perf_counter();
        sounds = dataset['testdata'].item()['sounds'];
        labels = dataset['testdata'].item()['labels'];
        val_sounds.extend(sounds);
        val_labels.extend(labels);

        valGen = ValGenerator(sounds, labels, opt);
        valX, valY = valGen.get_data();

        # print('{}/test_data_20K'.format(test_data_dir, s));
        np.savez_compressed('../acdnet_trained_data/test_data_20K', x=valX, y=valY);
        print('test with shape x{} and y{} took {:.2f} secs'.format(valX.shape, valY.shape, time.perf_counter()-start_time));
        sys.stdout.flush();

In [26]:
create_test_npz("../acdnet_training_data/single_fold/Src_test_fsd50_20K_data.npz");

+------------------------------+
| ACDNet_TL_Model_Extend Sound classification
+------------------------------+
| dataset  : uec_iot
| nEpochs  : 10
| LRInit   : 0.1
| schedule : [0.3, 0.6, 0.9]
| warmup   : 10
| batchSize: 88
| Splits: [1, 2, 3, 4, 5]
+------------------------------+
88
sounds shape:(88, 4, 30225)
(352, 1, 30225, 1)
(352, 12)


FileNotFoundError: [Errno 2] No such file or directory: '../acdnet_trained_data/test_data_20K.npz'